In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import tkinter as tk
from tkinter import ttk
from matplotlib.colors import ListedColormap

class WeaveGeneratorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Weave Pattern Generator")
        
        # Variables
        self.weave_types = ["Plain", "Basket", "Twill", "Satin", "Herringbone"]
        self.selected_weave = tk.StringVar(value="Plain")
        self.rows = tk.IntVar(value=10)
        self.cols = tk.IntVar(value=10)
        self.twill_a = tk.IntVar(value=2)
        self.twill_b = tk.IntVar(value=1)
        self.basket_n = tk.IntVar(value=2)
        self.satin_n = tk.IntVar(value=5)
        
        # GUI Layout
        self.setup_ui()
    
    def setup_ui(self):
        # Control Frame
        control_frame = ttk.Frame(self.root, padding="10")
        control_frame.pack(side=tk.LEFT, fill=tk.Y)
        
        # Weave Type Dropdown
        ttk.Label(control_frame, text="Weave Type:").pack()
        weave_combo = ttk.Combobox(
            control_frame, 
            textvariable=self.selected_weave, 
            values=self.weave_types,
            state="readonly"
        )
        weave_combo.pack(pady=5)
        weave_combo.bind("<<ComboboxSelected>>", self.update_controls)
        
        # Rows/Cols Sliders
        ttk.Label(control_frame, text="Rows:").pack()
        ttk.Scale(control_frame, from_=5, to=20, variable=self.rows, command=self.update_plot).pack()
        ttk.Label(control_frame, text="Columns:").pack()
        ttk.Scale(control_frame, from_=5, to=20, variable=self.cols, command=self.update_plot).pack()
        
        # Dynamic Controls (Twill/Basket/Satin)
        self.dynamic_frame = ttk.Frame(control_frame)
        self.dynamic_frame.pack(pady=10)
        self.update_controls()
        
        # Generate Button
        ttk.Button(control_frame, text="Generate", command=self.update_plot).pack(pady=10)
        
        # Plot Frame
        self.fig = Figure(figsize=(6, 6))
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.root)
        self.canvas.get_tk_widget().pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        # Initial Plot
        self.update_plot()
    
    def update_controls(self, *args):
        # Clear dynamic controls
        for widget in self.dynamic_frame.winfo_children():
            widget.destroy()
        
        # Add controls based on weave type
        weave_type = self.selected_weave.get()
        if weave_type == "Twill":
            ttk.Label(self.dynamic_frame, text="Twill (a/b):").pack()
            ttk.Scale(self.dynamic_frame, from_=1, to=5, variable=self.twill_a, command=self.update_plot).pack()
            ttk.Scale(self.dynamic_frame, from_=1, to=5, variable=self.twill_b, command=self.update_plot).pack()
        elif weave_type == "Basket":
            ttk.Label(self.dynamic_frame, text="Basket (n):").pack()
            ttk.Scale(self.dynamic_frame, from_=2, to=5, variable=self.basket_n, command=self.update_plot).pack()
        elif weave_type == "Satin":
            ttk.Label(self.dynamic_frame, text="Satin (n):").pack()
            ttk.Scale(self.dynamic_frame, from_=5, to=8, variable=self.satin_n, command=self.update_plot).pack()
    
    def generate_weave_pattern(self):
        weave_type = self.selected_weave.get().lower()
        rows, cols = self.rows.get(), self.cols.get()
        grid = np.zeros((rows, cols), dtype=int)
        
        if weave_type == "plain":
            for i in range(rows):
                for j in range(cols):
                    grid[i,j] = (i + j) % 2
        
        elif weave_type == "basket":
            n = self.basket_n.get()
            for i in range(rows):
                for j in range(cols):
                    grid[i,j] = (i // n + j // n) % 2
        
        elif weave_type == "twill":
            a, b = self.twill_a.get(), self.twill_b.get()
            for i in range(rows):
                for j in range(cols):
                    grid[i,j] = (i + (a - 1) * j) % (a + b) < a
        
        elif weave_type == "satin":
            n = self.satin_n.get()
            for i in range(rows):
                for j in range(cols):
                    if n == 5:
                        grid[i,j] = (i + 2 * j) % n == 0
                    elif n == 8:
                        grid[i,j] = (i + 3 * j) % n == 0
        
        elif weave_type == "herringbone":
            period = 4
            for i in range(rows):
                for j in range(cols):
                    if (j // period) % 2 == 0:
                        grid[i,j] = (i + j) % 4 < 2
                    else:
                        grid[i,j] = (i - j) % 4 < 2
        
        return grid
    
    def update_plot(self, *args):
        grid = self.generate_weave_pattern()
        self.ax.clear()
        
        # Plot grid
        cmap = ListedColormap(['white', 'black'])
        self.ax.imshow(grid, cmap=cmap, interpolation='nearest')
        
        # Add black borders
        for i in range(grid.shape[0] + 1):
            self.ax.axhline(i - 0.5, color='black', linewidth=1)
        for j in range(grid.shape[1] + 1):
            self.ax.axvline(j - 0.5, color='black', linewidth=1)
        
        self.ax.set_title(f"{self.selected_weave.get()} Weave")
        self.ax.axis('off')
        self.canvas.draw()

# Run the app
if __name__ == "__main__":
    root = tk.Tk()
    app = WeaveGeneratorApp(root)
    root.mainloop()